# LLM Optimization Modelling Experiment

In [30]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [104]:
problem = '''You are a city planner, looking to open facilities at some locations. We have a set of customers and a set of possible locations for opening facilities. Each potential location for establishing a facility incurs a fixed annual activation cost, which must be paid if the facility is used, regardless of the service volume it handles. Furthermore, this service volume at each facility is capped at a maximum annual limit. Additionally, there are transportation costs associated with servicing each customer from each facility.
The goal is to minimize the overall costs, which include both the fixed activation costs for any opened facilities and the transportation costs for servicing customers. This must be done while making sure that each customer’s demand is met, each facility does not exceed its maximum service volume, and customers can of course only be serviced by a facility that is opened. 
Please formulate this as a mathematical optimization model.
'''

## 2. Generate the mathematical model

In [113]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [114]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Facility Location

### Sets and Indices:

*   **I:** Set of customers (i ∈ I)
*   **J:** Set of potential facility locations (j ∈ J)

### Parameters:

*   **f<sub>j</sub>:** Fixed annual activation cost for facility at location j
*   **c<sub>ij</sub>:** Transportation cost per unit of demand for serving customer i from facility j
*   **d<sub>i</sub>:** Annual demand of customer i
*   **u<sub>j</sub>:** Maximum annual service volume capacity of facility at location j 

### Decision Variables:

*   **x<sub>ij</sub>:** Fraction of customer i's demand served by facility at location j (0 ≤ x<sub>ij</sub> ≤ 1)
*   **y<sub>j</sub>:** Binary variable indicating whether a facility is opened at location j (y<sub>j</sub> = 1 if open, 0 otherwise)

### Objective Function: 

Minimize the total cost (fixed activation costs + transportation costs)

```
Minimize Z = ∑_{j∈J} f_j * y_j + ∑_{i∈I} ∑_{j∈J} c_ij * d_i * x_ij
```

### Constraints:

1.  **Demand Satisfaction:** Each customer's demand must be fully met.

    ```
    ∑_{j∈J} x_ij = 1, ∀ i ∈ I
    ```
2.  **Capacity Restriction:** The service volume at each facility cannot exceed its capacity.

    ```
    ∑_{i∈I} d_i * x_ij ≤ u_j * y_j, ∀ j ∈ J 
    ```
3.  **Facility Usage:** A customer can only be served by an open facility.

    ```
    x_ij ≤ y_j, ∀ i ∈ I, ∀ j ∈ J
    ```
4.  **Binary and Non-negativity:**

    ```
    x_ij ≥ 0, ∀ i ∈ I, ∀ j ∈ J
    y_j ∈ {0, 1}, ∀ j ∈ J
    ```

This formulation represents a **mixed-integer linear programming (MILP)** model. Solving this model will provide the optimal location of facilities and allocation of customers to facilities, minimizing the total cost while satisfying all constraints. 


## 3. Generate the pyomo code

In [115]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [116]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
# Import Pyomo library
from pyomo.environ import *

# Create a concrete model
model = ConcreteModel()

# Sample Data (replace with your actual data)
I = {1, 2, 3}  # Set of customers
J = {1, 2}  # Set of potential facility locations
f = {1: 1000, 2: 1200}  # Fixed annual activation cost for each facility
c = {(1, 1): 5, (1, 2): 7, (2, 1): 6, (2, 2): 4, (3, 1): 8, (3, 2): 3}  # Transportation cost per unit of demand
d = {1: 200, 2: 300, 3: 250}  # Annual demand of each customer
u = {1: 500, 2: 600}  # Maximum annual service volume capacity of each facility

# Define sets
model.I = Set(initialize=I)
model.J = Set(initialize=J)

# Define parameters
model.f = Param(model.J, initialize=f)
model.c = Param(model.I, model.J, initialize=c)
model.d = Param(model.I, initialize=d)
model.u = Param(model.J, initialize=u)

# Define decision variables
model.x = Var(model.I, model.J, bounds=(0, 1))  # Fraction of customer demand served by each facility
model.y = Var(model.J, within=Binary)  # Whether a facility is opened

# Define objective function
def total_cost(model):
    fixed_cost = sum(model.f[j] * model.y[j] for j in model.J)
    transportation_cost = sum(model.c[i, j] * model.d[i] * model.x[i, j] for i in model.I for j in model.J)
    return fixed_cost + transportation_cost

model.objective = Objective(rule=total_cost, sense=minimize)

# Define constraints
def demand_satisfaction(model, i):
    return sum(model.x[i, j] for j in model.J) == 1

model.demand_constraint = Constraint(model.I, rule=demand_satisfaction)

def capacity_restriction(model, j):
    return sum(model.d[i] * model.x[i, j] for i in model.I) <= model.u[j] * model.y[j]

model.capacity_constraint = Constraint(model.J, rule=capacity_restriction)

def facility_usage(model, i, j):
    return model.x[i, j] <= model.y[j]

model.usage_constraint = Constraint(model.I, model.J, rule=facility_usage)
``` 
***
**Explanation:**

1. **Import Pyomo library:** Imports the necessary Pyomo library for creating optimization models.
2. **Create a concrete model:** Defines a concrete Pyomo model instance.
3. **Sample Data:**  Provides sample data for sets, parameters, and indices. Replace this with your actual data.
4. **Define sets:** Defines sets for customers (I) and potential facility locations (J).
5. **Define parameters:** Defines parameters including fixed activation costs, transportation costs, customer demands, and facility capacities.
6. **Define decision variables:** 
    *   `model.x`:  Fraction of customer demand served by each facility (continuous variable between 0 and 1).
    *   `model.y`: Binary variable indicating whether a facility is opened (1) or not (0). 
7. **Define objective function:** Defines the objective function to minimize the total cost, including fixed activation costs and transportation costs.
8. **Define constraints:** 
    *   **Demand Satisfaction:** Ensures that each customer's demand is fully met by summing the fractions served by all facilities.
    *   **Capacity Restriction:** Ensures that the total demand served by a facility does not exceed its capacity, considering whether it's open or closed.
    *   **Facility Usage:** Ensures a customer can only be served by an open facility. 

**To solve the model:**

```python
# Choose a solver (e.g., CBC, Gurobi, CPLEX)
solver = SolverFactory('cbc')  # Replace with your preferred solver

# Solve the model
results = solver.solve(model)

# Print results
print(results)

# Access specific results
for j in model.J:
    if model.y[j].value > 0.5:  # Check if facility j is open
        print(f"Facility at location {j} is open")
        for i in model.I:
            print(f"  Customer {i} demand served: {model.x[i, j].value:.2f}")
```

**Important Notes:**

*   Replace the sample data with your actual data.
*   Choose an appropriate solver based on your problem size and license availability.
*   Install the chosen solver and ensure it is accessible by Pyomo.


## 4. Input problem data and try running the generated code

In [118]:
# Import Pyomo library
from pyomo.environ import *

# Create a concrete model
model = ConcreteModel()

# Sample Data (replace with your actual data)
I = {1, 2, 3,4,5}  # Set of customers
J = {1, 2,3}  # Set of potential facility locations
f = {1: 1000, 2: 1000, 3: 1000}  # Fixed annual activation cost for each facility
c = {
    (1, 1): 4,  (1, 2): 6,  (1, 3): 9,
    (2, 1): 5,  (2, 2): 4,  (2, 3): 7,
    (3, 1): 6,  (3, 2): 3,  (3, 3): 4,
    (4, 1): 8,  (4, 2): 5,  (4, 3): 3,
    (5, 1): 10, (5, 2): 8,  (5, 3): 4,
}  # Transportation cost per unit of demand
d = {1: 80, 2: 270, 3: 250, 4: 160, 5: 180} # Annual demand of each customer
u = {1: 500, 2: 500, 3: 500}  # Maximum annual service volume capacity of each facility

# Define sets
model.I = Set(initialize=I)
model.J = Set(initialize=J)

# Define parameters
model.f = Param(model.J, initialize=f)
model.c = Param(model.I, model.J, initialize=c)
model.d = Param(model.I, initialize=d)
model.u = Param(model.J, initialize=u)

# Define decision variables
model.x = Var(model.I, model.J, bounds=(0, 1))  # Fraction of customer demand served by each facility
model.y = Var(model.J, within=Binary)  # Whether a facility is opened

# Define objective function
def total_cost(model):
    fixed_cost = sum(model.f[j] * model.y[j] for j in model.J)
    transportation_cost = sum(model.c[i, j] * model.d[i] * model.x[i, j] for i in model.I for j in model.J)
    return fixed_cost + transportation_cost

model.objective = Objective(rule=total_cost, sense=minimize)

# Define constraints
def demand_satisfaction(model, i):
    return sum(model.x[i, j] for j in model.J) == 1

model.demand_constraint = Constraint(model.I, rule=demand_satisfaction)

def capacity_restriction(model, j):
    return sum(model.d[i] * model.x[i, j] for i in model.I) <= model.u[j] * model.y[j]

model.capacity_constraint = Constraint(model.J, rule=capacity_restriction)

def facility_usage(model, i, j):
    return model.x[i, j] <= model.y[j]

model.usage_constraint = Constraint(model.I, model.J, rule=facility_usage)

# Choose a solver (e.g., CBC, Gurobi, CPLEX)
solver = SolverFactory('glpk')  # Replace with your preferred solver

# Solve the model
results = solver.solve(model)

# Print results
print(results)

# Access specific results
for j in model.J:
    if model.y[j].value > 0.5:  # Check if facility j is open
        print(f"Facility at location {j} is open")
        for i in model.I:
            print(f"  Customer {i} demand served: {model.x[i, j].value:.2f}")

(type: set).  This WILL potentially lead to nondeterministic behavior in Pyomo
(type: set).  This WILL potentially lead to nondeterministic behavior in Pyomo

Problem: 
- Name: unknown
  Lower bound: 5610.0
  Upper bound: 5610.0
  Number of objectives: 1
  Number of constraints: 23
  Number of variables: 18
  Number of nonzeros: 63
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.04089546203613281
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Facility at location 2 is open
  Customer 1 demand served: 1.00
  Customer 2 demand served: 1.00
  Customer 3 demand served: 0.60
  Customer 4 demand served: 0.00
  Customer 5 demand served: 0.00
Facility at location 3 is open
  Customer 1 demand served: -0.00
  Customer 2 demand served: -0.00
  Customer 3 demand served: 0.40
  Customer 4 demand served: 1.00
  

## 5. Correct the code to verify model viability (optional)

In [ ]:
#The model formulation above is unexpected, however still correct!

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [119]:
print(response.text)

## Mathematical Optimization Model for Facility Location

### Sets and Indices:

*   **I:** Set of customers (i ∈ I)
*   **J:** Set of potential facility locations (j ∈ J)

### Parameters:

*   **f<sub>j</sub>:** Fixed annual activation cost for facility at location j
*   **c<sub>ij</sub>:** Transportation cost per unit of demand for serving customer i from facility j
*   **d<sub>i</sub>:** Annual demand of customer i
*   **u<sub>j</sub>:** Maximum annual service volume capacity of facility at location j 

### Decision Variables:

*   **x<sub>ij</sub>:** Fraction of customer i's demand served by facility at location j (0 ≤ x<sub>ij</sub> ≤ 1)
*   **y<sub>j</sub>:** Binary variable indicating whether a facility is opened at location j (y<sub>j</sub> = 1 if open, 0 otherwise)

### Objective Function: 

Minimize the total cost (fixed activation costs + transportation costs)

```
Minimize Z = ∑_{j∈J} f_j * y_j + ∑_{i∈I} ∑_{j∈J} c_ij * d_i * x_ij
```

### Constraints:

1.  **Demand Satisfac

In [120]:
print(response2.text)

```python
# Import Pyomo library
from pyomo.environ import *

# Create a concrete model
model = ConcreteModel()

# Sample Data (replace with your actual data)
I = {1, 2, 3}  # Set of customers
J = {1, 2}  # Set of potential facility locations
f = {1: 1000, 2: 1200}  # Fixed annual activation cost for each facility
c = {(1, 1): 5, (1, 2): 7, (2, 1): 6, (2, 2): 4, (3, 1): 8, (3, 2): 3}  # Transportation cost per unit of demand
d = {1: 200, 2: 300, 3: 250}  # Annual demand of each customer
u = {1: 500, 2: 600}  # Maximum annual service volume capacity of each facility

# Define sets
model.I = Set(initialize=I)
model.J = Set(initialize=J)

# Define parameters
model.f = Param(model.J, initialize=f)
model.c = Param(model.I, model.J, initialize=c)
model.d = Param(model.I, initialize=d)
model.u = Param(model.J, initialize=u)

# Define decision variables
model.x = Var(model.I, model.J, bounds=(0, 1))  # Fraction of customer demand served by each facility
model.y = Var(model.J, within=Binary)  #